### Face Recognition Prediction

In [ ]:
import cv2
import numpy as np
import sqlite3
from datetime import datetime

# Initialize databases
def init_databases():
    # Database for known faces (staff, faculty, students)
    conn_known = sqlite3.connect('known_faces.db')
    c_known = conn_known.cursor()
    c_known.execute('''CREATE TABLE IF NOT EXISTS known_faces
                       (id INTEGER PRIMARY KEY AUTOINCREMENT,
                        name TEXT NOT NULL,
                        role TEXT NOT NULL,
                        encoding BLOB NOT NULL,
                        last_access TIMESTAMP)''')
    conn_known.commit()

    # Database for unknown faces (visitors, parents)
    conn_unknown = sqlite3.connect('unknown_faces.db')
    c_unknown = conn_unknown.cursor()
    c_unknown.execute('''CREATE TABLE IF NOT EXISTS unknown_faces
                         (id INTEGER PRIMARY KEY AUTOINCREMENT,
                          encoding BLOB NOT NULL,
                          gender TEXT,
                          first_seen TIMESTAMP)''')
    conn_unknown.commit()

    return conn_known, conn_unknown

# Store known faces (staff, faculty, students)
def store_known_face(conn, name, role, encoding):
    c = conn.cursor()
    c.execute("INSERT INTO known_faces (name, role, encoding, last_access) VALUES (?, ?, ?, ?)",
              (name, role, encoding.tobytes(), datetime.now()))
    conn.commit()

# Store unknown faces (visitors, parents)
def store_unknown_face(conn, encoding, gender):
    c = conn.cursor()
    c.execute("INSERT INTO unknown_faces (encoding, gender, first_seen) VALUES (?, ?, ?)",
              (encoding.tobytes(), gender, datetime.now()))
    conn.commit()

# Load known faces from database
def load_known_faces(conn):
    c = conn.cursor()
    c.execute("SELECT name, role, encoding FROM known_faces")
    return [(row[0], row[1], np.frombuffer(row[2], dtype=np.float64)) for row in c.fetchall()]

# Load gender detection model
gender_net = cv2.dnn.readNetFromCaffe('deploy_gender.prototxt', 'gender_net.caffemodel')
gender_list = ['Male', 'Female']

# Predict gender from face ROI
def predict_gender(face_roi):
    blob = cv2.dnn.blobFromImage(face_roi, 1.0, (227, 227), (104, 117, 123), swapRB=False)
    gender_net.setInput(blob)
    gender_preds = gender_net.forward()
    return gender_list[np.argmax(gender_preds)]

# Initialize Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Main function for real-time face recognition and gender detection
def main():
    # Initialize databases
    conn_known, conn_unknown = init_databases()

    # Load known faces
    known_faces = load_known_faces(conn_known)
    known_encodings = [enc for (name, role, enc) in known_faces]
    known_names = [name for (name, role, enc) in known_faces]
    known_roles = [role for (name, role, enc) in known_faces]

    # Initialize webcam
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        # Convert frame to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)

        for (x, y, w, h) in faces:
            face_roi = frame[y:y + h, x:x + w]

            # Predict gender
            gender = predict_gender(face_roi)

            # Convert face to RGB for face recognition
            rgb_face = cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)
            face_encoding = face_recognition.face_encodings(rgb_face)

            if len(face_encoding) > 0:
                face_encoding = face_encoding[0]

                # Compare with known faces
                matches = face_recognition.compare_faces(known_encodings, face_encoding)
                name = "Unknown"
                role = "Visitor"

                if True in matches:
                    match_index = matches.index(True)
                    name = known_names[match_index]
                    role = known_roles[match_index]
                    color = (0, 255, 0)  # Green for recognized faces
                else:
                    # Store unknown face in database
                    store_unknown_face(conn_unknown, face_encoding, gender)
                    color = (0, 0, 255)  # Red for unknown faces

                # Draw rectangle and label
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                label = f"{name} ({role}, {gender})" if name != "Unknown" else f"Unknown ({gender})"
                cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

        # Display the frame
        cv2.imshow('College Face Recognition', frame)

        # Exit on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()
    conn_known.close()
    conn_unknown.close()

if __name__ == "__main__":
    main()